In [1]:
import scipy.io as sio
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import timeit
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = (10,15)
ds = sio.loadmat('face.mat')

In [2]:
X = ds['X']
l = ds['l'][0]
split = 0.8
W = 46
H = 56
IDs = 52
X = pd.DataFrame(X.transpose())
N = X.shape[0]*split
l = pd.DataFrame(l)

#bagging 
bag_ratio = 0.6

In [14]:
X_train, X_test, l_train, l_test = train_test_split(X, l, test_size=(1-split), stratify = l)
# X_bar = np.mean(X, axis=1)
X_train, X_test = X_train.values, X_test.values
l_train, l_test = l_train.values, np.transpose(l_test.values)
X_bar = np.transpose([np.mean(X_train, axis=0)])
X_bar = np.ndarray.flatten(X_bar)
print('    l_train.shape',l_train.shape)
print('    X_train.shape', X_train.shape)

    l_train.shape (416, 1)
    X_train.shape (416, 2576)


## Functions to apply 1.3_a to bagging

In [4]:
from random import randrange
#Take the current split data and bag randomly 
def make_bag(X_data, l_data, ratio):
    X_bag = []
    l_bag = []
    bag_size = round(len(X_data)*ratio)
#     print(bag, bag_size)
    while len(X_bag) < bag_size:
        index = randrange(len(X_data))
        X_bag.append(X_data[index])
        l_bag.append(l_data[index])
    return np.asarray(X_bag), np.asarray(l_bag)

In [8]:
def bag_train_space(X_bag, l_bag):

    train_subspaces = []
    
    for i in range(round(IDs*bag_ratio)):
        images = []
        for j in range(int(N)):
            if l_train[j][0] == i+1:
                images.append(X_train[j])
        train_subspaces.append(images)
        
    return np.asarray(train_subspaces)

In [21]:
def SWSB(train_subspaces):
    elements = np.asarray(train_subspaces[0])
    elements_mean = np.mean(elements, axis=0)
    elements = np.subtract(elements,elements_mean)
    S_W = np.matmul(elements.T,elements)
    mean_diff = np.atleast_2d(elements_mean)-X_bar
    S_B = np.matmul(mean_diff.T, mean_diff)
    S = np.matmul(elements,np.transpose(elements))
    w, v = np.linalg.eig(S)
    U = preprocessing.normalize(np.matmul(np.transpose(elements),v), axis=0)
    for i in range(1,round(IDs*bag_ratio)):
        elements = np.asarray(train_subspaces[i])
        elements_mean = np.mean(elements, axis=0)
        elements = np.subtract(elements,elements_mean)
        S_W += np.matmul(elements.T,elements)
        mean_diff = np.atleast_2d(elements_mean)-X_bar
        S_B += np.matmul(mean_diff.T, mean_diff)
        S = np.matmul(elements,np.transpose(elements))
        w, v = np.linalg.eig(S)
        U = preprocessing.normalize(np.matmul(np.transpose(elements),v), axis=0)
    
    return S_W, S_B, w, v, U
                   

#### Calling Functions 
***For loop***
- make bags
- make training subspaces
- make eigen subspaces & S_W & S_B 
- PCA 
- LDA 
- W_opt 
- Error calculation
- Committie machine


In [23]:
# TEST SCRIPT
X_bag_0, l_bag_0 = make_bag(X_train, l_train, bag_ratio)

print('    X bag shape', X_bag_0.shape,'l bag shape', l_bag_0.shape)

bag_0_train_space = bag_train_space(X_bag_0, l_bag_0)

print('    bag train space shape:', bag_0_train_space.shape)

S_W_0, S_B_0, w_0, v_0, U_0 = SWSB(bag_0_train_space)

print('    S_W', S_W_0.shape, 'S_B', S_B_0.shape, 'w_0', w_0.shape, 'v_0', v_0.shape, 'U_0', U_0.shape)

    X bag shape (250, 2576) l bag shape (250, 1)
    bag train space shape: (31, 8, 2576)
    S_W (2576, 2576) S_B (2576, 2576) w_0 (8,) v_0 (8, 8) U_0 (2576, 8)
